In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [9]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-price aok-align-center'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price
# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [10]:
if __name__ == '__main__':
    
    #add your user agent
    HEADERS = ({'User-Agent':'','Accept-Language': 'en-US, en;q=0.5'})
    
    #webpage URL
    URL = "https://www.amazon.com/s?k=hair+blowout+brush+dryer&crid=2H54E2BN5KID4&sprefix=hair+blowout%2Caps%2C204&ref=nb_sb_ss_ts-doa-p_1_12"
    
    #HTTP Request
    webpage = requests.get(URL, headers=HEADERS)
    
    #soup object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")
    
    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        webpage2 = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        soup2 = BeautifulSoup(webpage2.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(soup2))
        d['price'].append(get_price(soup2))
        d['rating'].append(get_rating(soup2))
        d['reviews'].append(get_review_count(soup2))
        d['availability'].append(get_availability(soup2))
        
        
    amazon_blowout_tools_df = pd.DataFrame.from_dict(d)
    amazon_blowout_tools_df['title'].replace('', np.nan, inplace=True)
    amazon_blowout_tools_df = amazon_blowout_tools_df.dropna(subset=['title'])
    amazon_blowout_tools_df.to_csv("amazon_blowout_tools_data.csv", header=True, index=False)


In [11]:
amazon_blowout_tools_df

,title,price,rating,reviews,availability
0,"Professional Blowout Hair Dryer Brush, Black G...",,4.4 out of 5 stars,"4,600 ratings",In Stock.
1,L'ANGE HAIR Le Volume 2-in-1 Titanium Brush Dr...,,4.5 out of 5 stars,"5,913 ratings",In Stock.
2,"Hair Dryer Brush Blow Dryer Brush in One, Hair...",,4.3 out of 5 stars,"2,449 ratings",In Stock.
3,"Hair Dryer Brush Set, IG INGLAM 4 in 1 Blowout...",,4.2 out of 5 stars,"1,651 ratings",In Stock.
4,REVLON One-Step Volumizer Enhanced 1.0 Hair Dr...,,4.6 out of 5 stars,"336,404 ratings",In Stock.
...,...,...,...,...,...
65,"Negative ion Hair Straightener Brush,Small Hai...",,4.0 out of 5 stars,277 ratings,In Stock.
66,L'ANGE HAIR Le Volume 2-in-1 Titanium Brush Dr...,,4.5 out of 5 stars,"5,913 ratings",In Stock.
67,"Hair Dryer Brush & Volumizer Hot Air Brush, 3-...",,4.0 out of 5 stars,"3,065 ratings",In Stock.
68,"Hair Trends 1.5 Inch Plasma Hot Air Brush,Hair...",,4.6 out of 5 stars,35 ratings,In Stock.
